In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from datetime import timedelta
import numpy as np

pd.set_option('display.max_columns', None)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
colors = sns.color_palette('pastel')

In [2]:
df_click = pd.read_csv('C:/Users/mj985/section5/TeamProject2/model_new_session_id.csv')
df_trans_new = pd.read_csv('C:/Users/mj985/section5/TeamProject2/csv/transaction_new.csv')
df_customer = pd.read_csv('C:/Users/mj985/section5/TeamProject2/csv/customer.csv')

In [3]:
df_trans_user = df_trans_new.merge(df_customer, on = 'customer_id', how = 'left')

In [4]:
df_click = df_click.rename(columns={'session_id': 'new_session_id'})
df_click['session_id'] = df_click['new_session_id'].apply(lambda x: x[:36])

In [5]:
# session_id 열을 기준으로 df와 df_trans_new를 병합
merged_df = pd.merge(df_click, df_trans_user, on='session_id', how='left')
merged_df['event_time'] = pd.to_datetime(merged_df['event_time'])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21130155 entries, 0 to 21130154
Data columns (total 43 columns):
 #   Column                  Dtype              
---  ------                  -----              
 0   new_session_id          object             
 1   event_name              object             
 2   event_time              datetime64[ns, UTC]
 3   event_id                object             
 4   traffic_source          object             
 5   product_id_x            float64            
 6   quantity_x              float64            
 7   item_price_x            float64            
 8   payment_status_x        object             
 9   search_keywords         object             
 10  promo_code_x            object             
 11  promo_amount_x          float64            
 12  duration_time           object             
 13  session_id              object             
 14  created_at              object             
 15  customer_id             float64            
 16

In [6]:
merged_df['first_join_date'] = pd.to_datetime(merged_df['first_join_date'])

In [7]:
merged_df = merged_df[['new_session_id', 'event_name', 'product_id_x', 'event_time', 'total_amount', 'traffic_source', 'payment_method', 'shipment_date_limit', 'payment_status_x','shipment_fee', 'search_keywords', 'promo_code_x', 'promo_amount_x', 'customer_id', 'first_join_date']]

In [8]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21130155 entries, 0 to 21130154
Data columns (total 15 columns):
 #   Column               Dtype              
---  ------               -----              
 0   new_session_id       object             
 1   event_name           object             
 2   product_id_x         float64            
 3   event_time           datetime64[ns, UTC]
 4   total_amount         float64            
 5   traffic_source       object             
 6   payment_method       object             
 7   shipment_date_limit  object             
 8   payment_status_x     object             
 9   shipment_fee         float64            
 10  search_keywords      object             
 11  promo_code_x         object             
 12  promo_amount_x       float64            
 13  customer_id          float64            
 14  first_join_date      datetime64[ns]     
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), float64(5), object(8)
memory usage: 2.4+ GB


In [9]:
time_df = merged_df[['customer_id', 'new_session_id', 'event_time', 'first_join_date', 'traffic_source']].drop_duplicates(subset='new_session_id', keep = 'first')

In [10]:
time_df['created_at'] = time_df['event_time'].dt.strftime('%Y-%m-%d')
time_df.head(5)

,customer_id,new_session_id,event_time,first_join_date,traffic_source,created_at
0,74433.0,00001f29-3ede-4445-95f0-0178934e32ba_1,2019-05-17 11:51:52.276396+00:00,2017-02-05,MOBILE,2019-05-17
1,74433.0,00001f29-3ede-4445-95f0-0178934e32ba_2,2019-05-17 19:54:53.276396+00:00,2017-02-05,MOBILE,2019-05-17
3,74433.0,00001f29-3ede-4445-95f0-0178934e32ba_3,2019-05-18 03:58:25.276396+00:00,2017-02-05,MOBILE,2019-05-18
4,74433.0,00001f29-3ede-4445-95f0-0178934e32ba_4,2019-05-18 12:02:44.276396+00:00,2017-02-05,MOBILE,2019-05-18
5,74433.0,00001f29-3ede-4445-95f0-0178934e32ba_5,2019-05-18 20:04:54.276396+00:00,2017-02-05,MOBILE,2019-05-18


In [11]:
time_df = time_df.groupby(['customer_id', 'created_at'])[['first_join_date','new_session_id']].first().reset_index()

In [12]:
time_df['created_at'] = pd.to_datetime(time_df['created_at'])

In [13]:
time_df['diff'] = time_df.groupby('customer_id')['created_at'].diff().shift(-1)

In [14]:
time_df.dropna(subset=['diff'], inplace=True)

In [15]:
time_df

,customer_id,created_at,first_join_date,new_session_id,diff
0,3.0,2018-08-25,2018-08-25,f03b6825-1e6e-4916-bf71-f94ecbcbaee5_7952132,24 days
1,3.0,2018-09-18,2018-08-25,36673b2f-940a-42b5-994d-1cce4796b55f_1792149,1 days
2,3.0,2018-09-19,2018-08-25,36673b2f-940a-42b5-994d-1cce4796b55f_1792150,1 days
3,3.0,2018-09-20,2018-08-25,36673b2f-940a-42b5-994d-1cce4796b55f_1792151,2 days
4,3.0,2018-09-22,2018-08-25,36673b2f-940a-42b5-994d-1cce4796b55f_1792152,13 days
...,...,...,...,...,...
4396082,99998.0,2022-01-07,2018-07-01,4fc4d81b-4b29-4aef-bd1f-263c373950d1_2625124,1 days
4396083,99998.0,2022-01-08,2018-07-01,4fc4d81b-4b29-4aef-bd1f-263c373950d1_2625125,1 days
4396084,99998.0,2022-01-09,2018-07-01,4fc4d81b-4b29-4aef-bd1f-263c373950d1_2625126,2 days
4396085,99998.0,2022-01-11,2018-07-01,4fc4d81b-4b29-4aef-bd1f-263c373950d1_2625127,1 days


In [16]:
time_df['customer_id'].nunique()

40389

In [17]:
time_df['diff'].describe()

count                       4345385
mean      7 days 22:42:19.406381713
std      27 days 12:24:20.249427593
min                 1 days 00:00:00
25%                 1 days 00:00:00
50%                 1 days 00:00:00
75%                 5 days 00:00:00
max              1080 days 00:00:00
Name: diff, dtype: object

In [18]:
# check_df에서 diff칼럼이 15일 이상인 행 추출
mask = time_df['diff'] > pd.Timedelta(days=5)
# target 열 생성
time_df['target'] = np.where(mask, 1, 0)

In [19]:
time_df['target'].value_counts(normalize=True)

target
0    0.777027
1    0.222973
Name: proportion, dtype: float64

In [20]:
(time_df.groupby('customer_id')['target'].sum()/time_df.groupby('customer_id')['target'].size() * 100).describe()

count    40389.000000
mean        35.645487
std         27.994407
min          0.000000
25%         17.508418
50%         24.279835
75%         43.902439
max        100.000000
Name: target, dtype: float64

In [21]:
# time_df.to_csv('target.csv')

In [22]:
merged_df.head(5)

,new_session_id,event_name,product_id_x,event_time,total_amount,traffic_source,payment_method,shipment_date_limit,payment_status_x,shipment_fee,search_keywords,promo_code_x,promo_amount_x,customer_id,first_join_date
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,NaN,2019-05-17 11:51:52.276396+00:00,1427802.0,MOBILE,Credit Card,2019-05-21T13:04:58.898492Z,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,33855.0,2019-05-17 19:54:53.276396+00:00,1427802.0,MOBILE,Credit Card,2019-05-21T13:04:58.898492Z,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,NaN,2019-05-17 19:55:12.276396+00:00,1427802.0,MOBILE,Credit Card,2019-05-21T13:04:58.898492Z,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,NaN,2019-05-18 03:58:25.276396+00:00,1427802.0,MOBILE,Credit Card,2019-05-21T13:04:58.898492Z,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,NaN,2019-05-18 12:02:44.276396+00:00,1427802.0,MOBILE,Credit Card,2019-05-21T13:04:58.898492Z,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05


In [23]:
# ADD_TO_CART COUNT
merged_df = merged_df.merge(merged_df[merged_df['event_name'] == 'ADD_TO_CART'].groupby('new_session_id')['event_time'].count(), on = 'new_session_id', how = 'left')
merged_df['event_time_y'] = merged_df['event_time_y'].fillna(0)
merged_df = merged_df.rename(columns={'event_time_y': 'ADD_TO_CART_COUNT'})

In [24]:
merged_df = merged_df.rename(columns={'event_time_x': 'event_time'})

In [25]:
# SEARCH COUNT
merged_df = merged_df.merge(merged_df[merged_df['event_name'] == 'SEARCH'].groupby('new_session_id')['event_time'].count(), on = 'new_session_id', how = 'left')
merged_df['event_time_y'] = merged_df['event_time_y'].fillna(0)
merged_df = merged_df.rename(columns={'event_time_y': 'SEARCH_COUNT'})
merged_df = merged_df.rename(columns={'event_time_x': 'event_time'})

In [26]:
# ADD_PROMO COUNT
merged_df = merged_df.merge(merged_df[merged_df['event_name'] == 'ADD_PROMO'].groupby('new_session_id')['event_time'].count(), on = 'new_session_id', how = 'left')
merged_df['event_time_y'] = merged_df['event_time_y'].fillna(0)
merged_df = merged_df.rename(columns={'event_time_y': 'ADD_PROMO_COUNT'})
merged_df = merged_df.rename(columns={'event_time_x': 'event_time'})

In [27]:
# BOOKING COUNT
merged_df = merged_df.merge(merged_df[merged_df['event_name'] == 'BOOKING'].groupby('new_session_id')['event_time'].count(), on = 'new_session_id', how = 'left')
merged_df['event_time_y'] = merged_df['event_time_y'].fillna(0)
merged_df = merged_df.rename(columns={'event_time_y': 'BOOKING_COUNT'})
merged_df = merged_df.rename(columns={'event_time_x': 'event_time'})

In [28]:
# ITEM_DETAIL COUNT
merged_df = merged_df.merge(merged_df[merged_df['event_name'] == 'ITEM_DETAIL'].groupby('new_session_id')['event_time'].count(), on = 'new_session_id', how = 'left')
merged_df['event_time_y'] = merged_df['event_time_y'].fillna(0)
merged_df = merged_df.rename(columns={'event_time_y': 'ITEM_DETAIL_COUNT'})
merged_df = merged_df.rename(columns={'event_time_x': 'event_time'})

In [29]:
# merged_df = merged_df.merge(merged_df[merged_df['event_name'] == 'SEARCH'].groupby('new_session_id')['event_time_x'].count(), on = 'new_session_id', how = 'left')
# merged_df.head(5)

In [30]:
# merged_df.to_csv('check_num_events.csv')

In [31]:
time_df.head(5)

,customer_id,created_at,first_join_date,new_session_id,diff,target
0,3.0,2018-08-25,2018-08-25,f03b6825-1e6e-4916-bf71-f94ecbcbaee5_7952132,24 days,1
1,3.0,2018-09-18,2018-08-25,36673b2f-940a-42b5-994d-1cce4796b55f_1792149,1 days,0
2,3.0,2018-09-19,2018-08-25,36673b2f-940a-42b5-994d-1cce4796b55f_1792150,1 days,0
3,3.0,2018-09-20,2018-08-25,36673b2f-940a-42b5-994d-1cce4796b55f_1792151,2 days,0
4,3.0,2018-09-22,2018-08-25,36673b2f-940a-42b5-994d-1cce4796b55f_1792152,13 days,1


In [32]:
# df_share = time_df.merge(df_click, on = 'new_session_id', how = 'left')

In [33]:
# df_share.to_csv('merge_to_share.csv')

In [34]:
# df_share.head(5)

In [35]:
merged_df['shipment_date_limit'] = pd.to_datetime(merged_df['shipment_date_limit'])

In [36]:
merged_df['limit_ship'] = merged_df[merged_df['event_name'] == 'BOOKING']['shipment_date_limit'] - merged_df[merged_df['event_name'] == 'BOOKING']['event_time']

In [37]:
merged_df = merged_df.drop(['shipment_date_limit'], axis = 1)

In [38]:
merged_df.head(10)

,new_session_id,event_name,product_id_x,event_time,total_amount,traffic_source,payment_method,payment_status_x,shipment_fee,search_keywords,promo_code_x,promo_amount_x,customer_id,first_join_date,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,NaN,2019-05-17 11:51:52.276396+00:00,1427802.0,MOBILE,Credit Card,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05,0.0,0.0,0.0,0.0,0.0,NaT
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,33855.0,2019-05-17 19:54:53.276396+00:00,1427802.0,MOBILE,Credit Card,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05,1.0,0.0,0.0,0.0,0.0,NaT
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,NaN,2019-05-17 19:55:12.276396+00:00,1427802.0,MOBILE,Credit Card,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05,1.0,0.0,0.0,0.0,0.0,NaT
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,NaN,2019-05-18 03:58:25.276396+00:00,1427802.0,MOBILE,Credit Card,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05,0.0,0.0,0.0,0.0,0.0,NaT
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,NaN,2019-05-18 12:02:44.276396+00:00,1427802.0,MOBILE,Credit Card,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05,0.0,0.0,0.0,0.0,0.0,NaT
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,NaN,2019-05-18 20:04:54.276396+00:00,1427802.0,MOBILE,Credit Card,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05,0.0,0.0,0.0,0.0,1.0,NaT
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,NaN,2019-05-19 04:07:20.276396+00:00,1427802.0,MOBILE,Credit Card,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05,0.0,0.0,0.0,0.0,0.0,NaT
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,NaN,2019-05-19 20:17:04.276396+00:00,1427802.0,MOBILE,Credit Card,Success,0.0,NaN,NaN,NaN,74433.0,2017-02-05,0.0,0.0,0.0,1.0,0.0,1 days 16:47:54.622096
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,NaN,2019-08-24 05:47:16.849738+00:00,339790.0,MOBILE,LinkAja,NaN,0.0,NaN,NaN,NaN,43805.0,2019-08-16,1.0,0.0,0.0,1.0,0.0,NaT
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,NaN,2019-08-24 05:59:38.849738+00:00,339790.0,MOBILE,LinkAja,NaN,0.0,NaN,NaN,NaN,43805.0,2019-08-16,1.0,0.0,0.0,1.0,0.0,NaT


In [39]:
merged_df['session_id'] = merged_df['new_session_id'].apply(lambda x: x[:36])

In [40]:
merged_df.loc[merged_df['event_name'] != 'BOOKING', 'total_amount'] = 0

In [41]:
merged_df[merged_df['total_amount']!= 0]['total_amount'].describe()

count    1.254578e+06
mean     1.181572e+06
std      1.728944e+06
min      1.089800e+04
25%      2.466450e+05
50%      4.558770e+05
75%      1.360496e+06
max      2.350449e+07
Name: total_amount, dtype: float64

In [42]:
max_amounts = merged_df.groupby('new_session_id')['total_amount'].max().reset_index()
merged_df = merged_df.merge(max_amounts, on='new_session_id', suffixes=('', '_max'))
merged_df.loc[merged_df['total_amount'] != merged_df['total_amount_max'], 'total_amount'] = merged_df['total_amount_max']
merged_df = merged_df.drop('total_amount_max', axis=1)

In [43]:
merged_df = merged_df.drop(['event_time', 'product_id_x', 'payment_status_x', 'search_keywords', 'first_join_date', 'promo_code_x'], axis=1)

In [44]:
merged_df.head(20)

,new_session_id,event_name,total_amount,traffic_source,payment_method,shipment_fee,promo_amount_x,customer_id,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,session_id
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,74433.0,0.0,0.0,0.0,0.0,0.0,NaT,00001f29-3ede-4445-95f0-0178934e32ba
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,0.0,MOBILE,Credit Card,0.0,NaN,74433.0,1.0,0.0,0.0,0.0,0.0,NaT,00001f29-3ede-4445-95f0-0178934e32ba
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,74433.0,1.0,0.0,0.0,0.0,0.0,NaT,00001f29-3ede-4445-95f0-0178934e32ba
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,74433.0,0.0,0.0,0.0,0.0,0.0,NaT,00001f29-3ede-4445-95f0-0178934e32ba
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,74433.0,0.0,0.0,0.0,0.0,0.0,NaT,00001f29-3ede-4445-95f0-0178934e32ba
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,0.0,MOBILE,Credit Card,0.0,NaN,74433.0,0.0,0.0,0.0,0.0,1.0,NaT,00001f29-3ede-4445-95f0-0178934e32ba
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,0.0,MOBILE,Credit Card,0.0,NaN,74433.0,0.0,0.0,0.0,0.0,0.0,NaT,00001f29-3ede-4445-95f0-0178934e32ba
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,1427802.0,MOBILE,Credit Card,0.0,NaN,74433.0,0.0,0.0,0.0,1.0,0.0,1 days 16:47:54.622096,00001f29-3ede-4445-95f0-0178934e32ba
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,339790.0,MOBILE,LinkAja,0.0,NaN,43805.0,1.0,0.0,0.0,1.0,0.0,NaT,00003eca-954b-4150-aee1-63fc62f395cf
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,339790.0,MOBILE,LinkAja,0.0,NaN,43805.0,1.0,0.0,0.0,1.0,0.0,NaT,00003eca-954b-4150-aee1-63fc62f395cf


In [45]:
max_amounts = merged_df.groupby('new_session_id')['limit_ship'].max().reset_index()
merged_df = merged_df.merge(max_amounts, on='new_session_id', suffixes=('', '_max'))
merged_df.loc[merged_df['limit_ship'] != merged_df['limit_ship_max'], 'limit_ship'] = merged_df['limit_ship_max']
merged_df = merged_df.drop('limit_ship_max', axis=1)

In [46]:
merged_df = merged_df.drop(['customer_id','session_id'], axis=1)

In [47]:
merged_df.head(20)

,new_session_id,event_name,total_amount,traffic_source,payment_method,shipment_fee,promo_amount_x,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaT
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaT
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,1.0,NaT
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,1427802.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,1.0,0.0,1 days 16:47:54.622096
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,5 days 11:05:51.122116
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,5 days 11:05:51.122116


In [48]:
merged_df['PAGE_VIEW'] = merged_df.groupby('new_session_id')['event_name'].count()

In [49]:
merged_df.head(20)

,new_session_id,event_name,total_amount,traffic_source,payment_method,shipment_fee,promo_amount_x,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT,NaN
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaT,NaN
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaT,NaN
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT,NaN
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT,NaN
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,1.0,NaT,NaN
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT,NaN
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,1427802.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,1.0,0.0,1 days 16:47:54.622096,NaN
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,5 days 11:05:51.122116,NaN
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,5 days 11:05:51.122116,NaN


In [50]:
# def func(row):
#     if row <= timedelta(days = 2):
#         return 1
#     elif row <= timedelta(days = 3):
#         return 2
#     elif row <= timedelta(days = 4):
#         return 3
#     elif row <= timedelta(days = 10):
#         return 4
#     else:
#         return False
# merged_df['limit_ship'] = merged_df['limit_ship'].apply(func)

In [52]:
merged_df.head(20)

,new_session_id,event_name,total_amount,traffic_source,payment_method,shipment_fee,promo_amount_x,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,NaN
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,False,NaN
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,False,NaN
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,NaN
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,NaN
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,1.0,False,NaN
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,NaN
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,1427802.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,1.0,0.0,1,NaN
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,4,NaN
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,4,NaN


In [53]:
merged_df['PAGE_VIEW'] = merged_df.groupby('new_session_id')['event_name'].transform('count')

In [54]:
merged_df.head(20)

,new_session_id,event_name,total_amount,traffic_source,payment_method,shipment_fee,promo_amount_x,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,1
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,False,2
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,False,2
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,1
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,1
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,1.0,False,1
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,1
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,1427802.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,1.0,0.0,1,1
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,4,4
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,4,4


In [55]:
merged_df.loc[merged_df['limit_ship'] == False, 'limit_ship'] = 0

In [56]:
merged_df.head(20)

,new_session_id,event_name,total_amount,traffic_source,payment_method,shipment_fee,promo_amount_x,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,0,2
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,0,2
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,1.0,0,1
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,1427802.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,1.0,0.0,1,1
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,4,4
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,4,4


In [57]:
merged_df.loc[merged_df['event_name'] != 'BOOKING', 'shipment_fee'] = 0

In [58]:
merged_df[merged_df['event_name'] == 'BOOKING']['shipment_fee'].describe()

count    1.254578e+06
mean     9.190831e+03
std      9.372016e+03
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+04
75%      1.000000e+04
max      5.000000e+04
Name: shipment_fee, dtype: float64

In [59]:
# def func(x):
#     if x <= 0:
#         return 0 
#     elif x <= 5000:
#         return 1
#     elif x <= 10000:
#         return 2
#     elif x <= 15000:
#         return 3
#     else:
#         return 4


# merged_df['shipment_fee'] = merged_df['shipment_fee'].apply(func)

In [60]:
merged_df.head(20)

,new_session_id,event_name,total_amount,traffic_source,payment_method,shipment_fee,promo_amount_x,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,0.0,MOBILE,Credit Card,0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,0.0,MOBILE,Credit Card,0,NaN,1.0,0.0,0.0,0.0,0.0,0,2
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,0.0,MOBILE,Credit Card,0,NaN,1.0,0.0,0.0,0.0,0.0,0,2
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,0.0,MOBILE,Credit Card,0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,0.0,MOBILE,Credit Card,0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,0.0,MOBILE,Credit Card,0,NaN,0.0,0.0,0.0,0.0,1.0,0,1
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,0.0,MOBILE,Credit Card,0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,1427802.0,MOBILE,Credit Card,0,NaN,0.0,0.0,0.0,1.0,0.0,1,1
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,339790.0,MOBILE,LinkAja,0,NaN,1.0,0.0,0.0,1.0,0.0,4,4
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,339790.0,MOBILE,LinkAja,0,NaN,1.0,0.0,0.0,1.0,0.0,4,4


In [61]:
merged_df['total_promo'] = merged_df.groupby('new_session_id')['promo_amount_x'].transform('sum')

In [62]:
merged_df['total_promo'].describe()

count    2.113016e+07
mean     2.151051e+03
std      1.157101e+04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.537180e+05
Name: total_promo, dtype: float64

In [63]:
# check_describe = merged_df.drop_duplicates(subset='new_session_id')

In [64]:
# check_describe[check_describe['total_amount'] > 0]['total_amount'].describe()

In [65]:
# def func(x):
#     if x <= 0:
#         return 0 
#     elif x <= 2.037932e+05:
#         return 1
#     elif x <= 3.029890e+05:
#         return 2
#     elif x <= 5.147018e+05:
#         return 3
#     else:
#         return 4


# merged_df['total_amount'] = merged_df['total_amount'].apply(func)

In [66]:
# check_describe[check_describe['total_promo'] > 0]['total_promo'].describe()

In [67]:
# def func(x):
#     if x <= 0:
#         return 0 
#     elif x <= 3721.000000:
#         return 1
#     elif x <= 5404.000000:
#         return 2
#     elif x <= 8089.500000:
#         return 3
#     else:
#         return 4


# merged_df['total_promo'] = merged_df['total_promo'].apply(func)

In [69]:
merged_df = merged_df.drop(['event_name', 'traffic_source', 'payment_method', 'promo_amount_x'], axis=1)

In [73]:
merged_df = merged_df.drop_duplicates(subset='new_session_id')

In [71]:
time_df = time_df.drop(['first_join_date', 'created_at', 'diff'], axis=1)

In [72]:
time_df.head(5)

,customer_id,new_session_id,target
0,3.0,f03b6825-1e6e-4916-bf71-f94ecbcbaee5_7952132,1
1,3.0,36673b2f-940a-42b5-994d-1cce4796b55f_1792149,0
2,3.0,36673b2f-940a-42b5-994d-1cce4796b55f_1792150,0
3,3.0,36673b2f-940a-42b5-994d-1cce4796b55f_1792151,0
4,3.0,36673b2f-940a-42b5-994d-1cce4796b55f_1792152,1


In [74]:
time_df = time_df.merge(merged_df, on = 'new_session_id', how = 'left')

In [77]:
time_df['limit_ship'] = time_df['limit_ship'].astype(int)

In [78]:
time_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4345385 entries, 0 to 4345384
Data columns (total 13 columns):
 #   Column             Dtype  
---  ------             -----  
 0   customer_id        float64
 1   new_session_id     object 
 2   target             int32  
 3   total_amount       int64  
 4   shipment_fee       int64  
 5   ADD_TO_CART_COUNT  float64
 6   SEARCH_COUNT       float64
 7   ADD_PROMO_COUNT    float64
 8   BOOKING_COUNT      float64
 9   ITEM_DETAIL_COUNT  float64
 10  limit_ship         int32  
 11  PAGE_VIEW          int64  
 12  total_promo        int64  
dtypes: float64(6), int32(2), int64(4), object(1)
memory usage: 397.8+ MB


In [85]:
time_df.head(10)

,customer_id,new_session_id,target,total_amount,shipment_fee,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW,total_promo
0,3.0,f03b6825-1e6e-4916-bf71-f94ecbcbaee5_7952132,1,4,0,2.0,0.0,1.0,1.0,0.0,1,8,3
1,3.0,36673b2f-940a-42b5-994d-1cce4796b55f_1792149,0,0,0,0.0,0.0,0.0,0.0,0.0,0,1,0
2,3.0,36673b2f-940a-42b5-994d-1cce4796b55f_1792150,0,0,0,2.0,0.0,0.0,0.0,0.0,0,2,0
3,3.0,36673b2f-940a-42b5-994d-1cce4796b55f_1792151,0,0,0,1.0,0.0,0.0,0.0,0.0,0,1,0
4,3.0,36673b2f-940a-42b5-994d-1cce4796b55f_1792152,1,1,2,0.0,0.0,0.0,1.0,0.0,3,1,0
5,3.0,93e1d3c4-bed1-4ef4-b79f-e24a79e905b7_4883279,0,0,0,0.0,0.0,0.0,0.0,0.0,0,1,0
6,3.0,93e1d3c4-bed1-4ef4-b79f-e24a79e905b7_4883280,0,0,0,1.0,1.0,0.0,0.0,0.0,0,2,0
7,3.0,93e1d3c4-bed1-4ef4-b79f-e24a79e905b7_4883281,0,0,0,1.0,0.0,1.0,0.0,0.0,0,2,4
8,3.0,93e1d3c4-bed1-4ef4-b79f-e24a79e905b7_4883283,0,0,0,1.0,0.0,0.0,0.0,0.0,0,1,0
9,3.0,93e1d3c4-bed1-4ef4-b79f-e24a79e905b7_4883284,0,0,0,0.0,0.0,0.0,0.0,0.0,0,1,0


In [101]:
time_df['total_session'] = time_df.groupby('customer_id')['new_session_id'].transform('count')

In [87]:
time_df.head(10)

,customer_id,new_session_id,target,total_amount,shipment_fee,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW,total_promo,total_session
0,3.0,f03b6825-1e6e-4916-bf71-f94ecbcbaee5_7952132,1,4,0,2.0,0.0,1.0,1.0,0.0,1,8,3,321
1,3.0,36673b2f-940a-42b5-994d-1cce4796b55f_1792149,0,0,0,0.0,0.0,0.0,0.0,0.0,0,1,0,321
2,3.0,36673b2f-940a-42b5-994d-1cce4796b55f_1792150,0,0,0,2.0,0.0,0.0,0.0,0.0,0,2,0,321
3,3.0,36673b2f-940a-42b5-994d-1cce4796b55f_1792151,0,0,0,1.0,0.0,0.0,0.0,0.0,0,1,0,321
4,3.0,36673b2f-940a-42b5-994d-1cce4796b55f_1792152,1,1,2,0.0,0.0,0.0,1.0,0.0,3,1,0,321
5,3.0,93e1d3c4-bed1-4ef4-b79f-e24a79e905b7_4883279,0,0,0,0.0,0.0,0.0,0.0,0.0,0,1,0,321
6,3.0,93e1d3c4-bed1-4ef4-b79f-e24a79e905b7_4883280,0,0,0,1.0,1.0,0.0,0.0,0.0,0,2,0,321
7,3.0,93e1d3c4-bed1-4ef4-b79f-e24a79e905b7_4883281,0,0,0,1.0,0.0,1.0,0.0,0.0,0,2,4,321
8,3.0,93e1d3c4-bed1-4ef4-b79f-e24a79e905b7_4883283,0,0,0,1.0,0.0,0.0,0.0,0.0,0,1,0,321
9,3.0,93e1d3c4-bed1-4ef4-b79f-e24a79e905b7_4883284,0,0,0,0.0,0.0,0.0,0.0,0.0,0,1,0,321


In [102]:
time_df['total_session'].describe()

count    4.345385e+06
mean     2.665084e+02
std      1.945406e+02
min      1.000000e+00
25%      1.120000e+02
50%      2.230000e+02
75%      3.850000e+02
max      1.330000e+03
Name: total_session, dtype: float64

In [103]:
# def func(x):
#     if x <= 1:
#         return 1
#     elif x <= 2:
#         return 2
#     elif x <= 4:
#         return 3
#     else:
#         return 4


# time_df['total_session'] = time_df['total_session'].apply(func)

In [98]:
time_df.to_csv('not_label_model.csv')

In [107]:
time_df['PAGE_VIEW'].describe()

count    4.345385e+06
mean     2.460672e+00
std      1.139106e+01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      3.483000e+03
Name: PAGE_VIEW, dtype: float64

In [108]:
def func(x):
    if x < 1:
        return 0
    elif x <= 1:
        return 1
    elif x <= 2:
        return 2
    else:
        return 3


time_df['PAGE_VIEW'] = time_df['PAGE_VIEW'].apply(func)

In [109]:
time_df['SEARCH_COUNT'].describe()

count    4.345385e+06
mean     1.752176e-01
std      1.278883e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.010000e+02
Name: SEARCH_COUNT, dtype: float64

In [110]:
def func(x):
    if x < 1:
        return 0
    elif x <= 1:
        return 1
    else:
        return 2


time_df['SEARCH_COUNT'] = time_df['SEARCH_COUNT'].apply(func)
time_df['ITEM_DETAIL_COUNT'] = time_df['ITEM_DETAIL_COUNT'].apply(func)
time_df['ADD_PROMO_COUNT'] = time_df['ADD_PROMO_COUNT'].apply(func)
time_df['ADD_TO_CART_COUNT'] = time_df['ADD_TO_CART_COUNT'].apply(func)

In [111]:
df = time_df.drop(['customer_id', 'new_session_id'], axis = 1)

In [112]:
df.corr()

,target,total_amount,shipment_fee,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW,total_promo,total_session
target,1.000000,0.499096,0.418415,-0.005549,-0.001492,0.028377,0.360712,-0.003206,0.506397,0.018098,0.025991,-0.042900
total_amount,0.499096,1.000000,0.684585,-0.060294,0.003292,0.006250,0.713817,-0.006365,0.825121,0.088617,0.003922,-0.019358
shipment_fee,0.418415,0.684585,1.000000,-0.136236,-0.084817,-0.057061,0.490242,-0.092403,0.682496,-0.060446,-0.054494,0.010200
ADD_TO_CART_COUNT,-0.005549,-0.060294,-0.136236,1.000000,0.126251,0.065041,-0.040400,0.125525,-0.063959,0.574923,0.058065,-0.035789
SEARCH_COUNT,-0.001492,0.003292,-0.084817,0.126251,1.000000,0.131001,0.001014,0.132898,0.002722,0.194529,0.121105,-0.016499
ADD_PROMO_COUNT,0.028377,0.006250,-0.057061,0.065041,0.131001,1.000000,0.009894,0.114161,0.002445,0.248806,0.926297,-0.017796
BOOKING_COUNT,0.360712,0.713817,0.490242,-0.040400,0.001014,0.009894,1.000000,-0.005091,0.597144,0.179812,0.006895,-0.013448
ITEM_DETAIL_COUNT,-0.003206,-0.006365,-0.092403,0.125525,0.132898,0.114161,-0.005091,1.000000,-0.007169,0.202564,0.104150,-0.018648
limit_ship,0.506397,0.825121,0.682496,-0.063959,0.002722,0.002445,0.597144,-0.007169,1.000000,0.026121,0.001761,-0.019312
PAGE_VIEW,0.018098,0.088617,-0.060446,0.574923,0.194529,0.248806,0.179812,0.202564,0.026121,1.000000,0.229661,-0.027806
